In [0]:
!rm -r east_icdar2015_resnet_v1_50_rbox/
!rm StraboDependency.zip
!rm install.sh
!rm open\?id\=17Tat38W_M_4yaHS_dHTf2TakNaT-jQel
!rm text-detection-requirements.txt

In [0]:
!rm -r strabo-text-recognition-deep-learning/
#!pip install gdown
!gdown https://drive.google.com/uc?id=1MjBMO_Ql7kth7VPPX6XkDXW4sr1qtjDV
!gdown https://drive.google.com/uc?id=17Tat38W_M_4yaHS_dHTf2TakNaT-jQel
!unzip StraboDependency.zip
!sh install.sh
!sudo apt-get install python3-dev
!sudo apt-get install software-properties-common
!sudo apt-get install libgeos-dev
!sudo pip3 install -r text-detection-requirements.txt
!sudo add-apt-repository -y ppa:alex-p/tesseract-ocr
!sudo apt-get update
!sudo apt install -y libtesseract-dev libleptonica-dev liblept5
!sudo apt install -y tesseract-ocr
!sudo pip3 install git+https://github.com/spatial-computing/tesserocr
!export LC_ALL=C
%cd strabo-text-recognition-deep-learning/
!wget "https://uob-my.sharepoint.com/:u:/g/personal/rs16175_bristol_ac_uk/ESYILU6Hu4dGoo3gfNr6rHUB75nM6zZjcTvie_3gbY74iA?download=1" --output-document "MapData.zip"
!unzip MapData.zip 

# Make These Changes in the files

## run_command_line.py => in main(), change the parser args to:

parser.add_argument('--image',default='c2.png')
parser.add_argument('--config',default='configuration.ini')

And also

checkpoint_path = '../east_icdar2015_resnet_v1_50_rbox'


## text_recognition.py => by PyTessBaseAPI()


PyTessBaseAPI(path='/usr/share/tesseract-ocr/4.00/tessdata',psm=PSM.AUTO_OSD, oem=OEM.LSTM_ONLY) as api:


In [0]:
import json 
import shutil
import os 
from math import radians, cos, sin, asin, sqrt,pi
import os
from pathlib import Path
from PIL import Image
import glob
from osgeo import gdal

city = "Oldham"
dataBasePath = "Sample/Raw/"+city

def findAllFileIds():
  prjFiles = list(filter(lambda x: x.endswith(".prj"),os.listdir(dataBasePath)))
  return list(map(lambda x: Path(x).stem,prjFiles))

def setup():
  fileIds = findAllFileIds()
  for name in fileIds:
    gdal.Translate(os.getcwd()+"/"+dataBasePath+"/"+name + '.png',
               os.getcwd()+"/"+dataBasePath+"/"+name + '.tif',
               options="-of PNG")

setup()

In [0]:
def getAllTextForCity():
  textTuples = []
  maps = findAllFileIds()
  # shutil.rmtree("static/results/test2")
  exceptedFiles = []
  for tile in maps:
    try:
      textInTile = retrieveTextWLatLonsFromImage(tile)
      textTuples.extend(textInTile)
    except Exception as e:
      print(e)
      exceptedFiles.append(tile)
    shutil.rmtree("static/results/test2")
  return textTuples, exceptedFiles

def retrieveTextWLatLonsFromImage(imageId):
  targetFile = dataBasePath+"/"+imageId
  targetFileImage = targetFile + ".png"
  targetFileGeo = targetFile + ".tfw"
  targetFileMeta = targetFile + ".prj"
  !python3 run_command_line.py --image=$targetFileImage
  textWLatLons = retrieveModelOutputStrabo(targetFileGeo,imageId)
  return textWLatLons

def retrieveModelOutputStrabo(targetFileGeo,imageId):
  basePath = "static/results/test2/"+dataBasePath+"/"
  topLevelName =  list(filter(lambda x: os.path.isdir(basePath+x),os.listdir(basePath)))[0]
  topLevelName = basePath+topLevelName
  with open(topLevelName+"/final.txt",'rb') as f:
    parsed = (json.load(f))
    
    with open(imageId+".json",'w',encoding='utf-8') as f:
      json.dump(parsed,f)

  bboxTextLocations = (locateTextPixelsInStraboJson(parsed))
  singleXY = list(map(lambda x: mapBboxToSingleCoord(x),bboxTextLocations))
  latLonPoints = list(map(lambda x: mapImagePixelToLatLon(x,targetFileGeo),singleXY))
  latLonPoints = list(map(lambda x: (x[0],x[1],imageId), latLonPoints))
  return latLonPoints

def locateTextPixelsInStraboJson(finalJson):
  featureList = finalJson["features"]
  # featuresOfInterest = list(filter(lambda x: isChimney(x["NameAfterDictionary"]),featureList))
  bboxes = []
  for entry in featureList:
    uniqueCoords = (list(set(list(map(lambda x: (x[0],x[1]),entry['geometry']['coordinates'][0])))))
    if len(uniqueCoords) == 4:
      bboxes.append((uniqueCoords,entry["NameAfterDictionary"]))
  
  return bboxes

def mapBboxToSingleCoord(textTuple):
  locations, text = textTuple 
  xs = list(map(lambda x: x[0],locations))
  ys = list(map(lambda x: x[1],locations))

  newX = (min(xs)+max(xs)) / 2
  newY = (min(ys)+max(ys)) / 2
  return ((newX,newY),text)

def isChimney(proposalString):
  return True
  chimneyKeys = ["chimney", "chi", "chy"]
  proposalString = proposalString.lower()
  validChimneys = len(list(filter(lambda x: x in proposalString,chimneyKeys)))
  return validChimneys > 0

def getBaseLatLon():
  return 53.9153862, -2.6224470139

def convertPixelToMetreOffset(coord,targetFileGeo):
  pixelX, pixelY = coord
  with open(targetFileGeo, 'rb') as f:
    lines = f.readlines()
    lonOffsetFromBase = float(lines[-1].strip())
    latOffsetFromBase = float(lines[-2].strip())
    yPixelsToMetres = 1 / float(lines[-3].strip())
    xPixelsToMetres = 1 / float(lines[0].strip())

    metreOffsetOnY = pixelY / yPixelsToMetres
    metreOffsetOnX = pixelX / xPixelsToMetres

    return (metreOffsetOnX+latOffsetFromBase,metreOffsetOnY+lonOffsetFromBase)

def mapImagePixelToLatLon(inbound,targetFileGeo):
  coord, text = inbound
  lat,lon = getBaseLatLon()
  Xm, Ym = convertPixelToMetreOffset(coord,targetFileGeo)
  newLatFromBase, newLonFromBase = transformLatLonByXMetres(lat,lon,Ym,Xm)
  return ((newLatFromBase, newLonFromBase),text)

def transformLatLonByXMetres(lat,lon,dn,de):
    R=6377563.396 #from their files
    dLat = dn/R
    dLon = de/(R*cos(pi*lat/180))
    latO = lat + dLat * 180/pi
    lonO = lon + dLon * 180/pi

    return (latO,lonO)

def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [0]:
chimneysOfBurnley = getAllTextForCity()

In [18]:
chimneysOfBurnley

([((53.78975454102456, -2.278330547288232), '', '1909064010241'),
  ((53.78973149128649, -2.278151928138554), '', '1909064010241'),
  ((53.789791306309255, -2.278524993451173), '', '1909064010241'),
  ((53.78981226061659, -2.280616097203463), 'park\n', '1909064010241'),
  ((53.789144770681, -2.2767084399289486), '', '1909064010241'),
  ((53.78987017070233, -2.2789313277916703), '', '1909064010241'),
  ((53.78898285103338, -2.2793835281706025), 'oe\n', '1909064010241'),
  ((53.789444798263354, -2.2783899793380344), '', '1909064010241'),
  ((53.78873997156195, -2.27784927688494), '447:\n\nto\n', '1909064010241'),
  ((53.78978330557372, -2.2806196502064404), 'folly\n', '1909064010241'),
  ((53.788700348871714, -2.278051152054106), ' \n', '1909064010241'),
  ((53.7886689174107, -2.2781480521353057), ' \n', '1909064010241'),
  ((53.78977873372485, -2.2802779159200757), ' \n', '1909064010241'),
  ((53.788790261899564, -2.2778263438657227), ' \n', '1909064010241'),
  ((53.78896189672604, -2.2

In [0]:
import pickle
with open('allWFileName','wb') as f:
  pickle.dump(chimneysOfBurnley,f)

In [0]:
import pickle
with open('allWFileName','rb') as f:
  print(pickle.load(f))
